In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import time

from src.infrastructure import acpds
from src.domain.engine import train_model
from src.model import rcnn, fast_rcnn_fpn

torch.cuda.empty_cache() # not recommanded to call by end user by pytorch

# torch.backends.cudnn.benchmark if True,  it enables benchmark mode in cudnn which is good whenever the input sizes for the network do not vary. This way, cudnn look for the optimal set of algorithms for that particular 
# configuration (which takes some time). This usually leads to faster runtime. But if your input sizes changes at each iteration, then cudnn will benchmark every time a new size appears, 
# possibly leading to worse runtime performances.
# torch.backends.cudnn.benchmark = True 

2025-11-12 10:19:57.670573: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-12 10:19:57.816262: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-11-12 10:19:58.605765: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
print( torch.cuda.is_available() )  # True si GPU détecté
print( torch.cuda.device_count() )  # nombre total de GPU disponibles

True
1


In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('gpu')
cpus = tf.config.list_physical_devices('cpu')
print(len(gpus), "GPU(s) detected")
print(len(cpus), "CPU(s) detected")
for gpu in gpus:
    print(gpu)

for cpu in cpus:
    print(cpu)

print(tf.test.is_built_with_cuda())

0 GPU(s) detected
0 CPU(s) detected
True


2025-11-12 10:20:48.316493: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-12 10:20:48.317810: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# download the dataset
data_url = "../../data/parking_rois_gopro"
print(os.listdir(data_url))

['annotations.json', 'images']


In [4]:
divice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# divice = "cpu"
print( f"Using device: {divice}" )

Using device: cuda


In [5]:
# load the dataset
train_ds, val_ds, test_ds = acpds.create_datasets(data_url)

train_ds_tf, val_ds_tf, test_ds_tf = acpds.create_datasets_tf(data_url)

# train_ds, val_ds, test_ds are instances of torch.utils.data.dataloader.DataLoader
# print length of each dataset
print(f"Length of TORCH / TF train dataset: {len(train_ds)}/{len(train_ds_tf)}")
print(f"Length of TORCH / TF val dataset: {len(val_ds)}/{len(val_ds_tf)}")
print(f"Length of TORCH / TF test dataset: {len(test_ds)}/{len(test_ds_tf)}")

Length of TORCH / TF train dataset: 231/231
Length of TORCH / TF val dataset: 35/35
Length of TORCH / TF test dataset: 27/27


In [7]:
# train the model
model = fast_rcnn_fpn.FasterRCNN_FPN()

/home/besttic-rd/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/besttic-rd/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/besttic-rd/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
model = model.to(torch.float32)
# train_ds = train_ds.to(torch.float32)
# val_ds   = train_ds.to(torch.float32)
# test_ds  = train_ds.to(torch.float32)

In [ ]:
now = time.time()
output_dir = "../../data/" + model._get_name() + "_" + str(now)
print(output_dir)

train_model(model, train_ds, val_ds, test_ds, output_dir, device=divice, epochs=10, verbose=True) 

../../data/FasterRCNN_FPN_1762791114.6586063


/home/besttic-rd/Documents/besttic/parkOccupancy/src/infrastructure/acpds.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float16)


In [8]:
now = time.time()
output_dir = "../../data/" + model._get_name() + "_" + str(now)
output_dir                                   

'../../data/FasterRCNN_FPN_1762939457.0602252'

In [ ]:
train_model(model, train_ds, val_ds, test_ds, output_dir, device=divice, epochs=10, verbose=True)

In [11]:
import torch
torch.cuda.empty_cache()

In [ ]:
from torch.amp import autocast

In [6]:
# now in string format

params = [p for p in model.parameters() if p.requires_grad]

In [13]:
for param in model.parameters() :
    #if not isinstance(parm, torch.nn.parameter.Parameter):
    print( param.shape )

torch.Size([64, 3, 7, 7])
torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([256, 64, 1, 1])
torch.Size([256, 64, 1, 1])
torch.Size([64, 256, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([256, 64, 1, 1])
torch.Size([64, 256, 1, 1])
torch.Size([64, 64, 3, 3])
torch.Size([256, 64, 1, 1])
torch.Size([128, 256, 1, 1])
torch.Size([128, 128, 3, 3])
torch.Size([512, 128, 1, 1])
torch.Size([512, 256, 1, 1])
torch.Size([128, 512, 1, 1])
torch.Size([128, 128, 3, 3])
torch.Size([512, 128, 1, 1])
torch.Size([128, 512, 1, 1])
torch.Size([128, 128, 3, 3])
torch.Size([512, 128, 1, 1])
torch.Size([128, 512, 1, 1])
torch.Size([128, 128, 3, 3])
torch.Size([512, 128, 1, 1])
torch.Size([256, 512, 1, 1])
torch.Size([256, 256, 3, 3])
torch.Size([1024, 256, 1, 1])
torch.Size([1024, 512, 1, 1])
torch.Size([256, 1024, 1, 1])
torch.Size([256, 256, 3, 3])
torch.Size([1024, 256, 1, 1])
torch.Size([256, 1024, 1, 1])
torch.Size([256, 256, 3, 3])
torch.Size([1024, 256, 1, 1])
torch.Size([256, 1024, 1,